# Langchain Basics

In [1]:
%%time
!pip install openai langchain langchain_community langchain_experimental langchain_openai pinecone-client python-dotenv tiktoken -Uq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.6/117.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.9/378.9 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275

In [2]:
!pip show openai langchain pinecone-client python-dotenv tiktoken

Name: openai
Version: 1.37.1
Summary: The official Python library for the openai API
Home-page: https://github.com/openai/openai-python
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: anyio, distro, httpx, pydantic, sniffio, tqdm, typing-extensions
Required-by: langchain-openai
---
Name: langchain
Version: 0.2.11
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-community
---
Name: pinecone-client
Version: 5.0.0
Summary: Pinecone client and SDK
Home-page: https://www.pinecone.io
Author: Pinecone Systems, Inc.
Author-email: support@pinecone.io
License: Apache-2.0
Location: /usr/local/lib/python3.10/dis

In [3]:
!pip install langchain_community -q

## Python Dot-Env

In [4]:
# vscode
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(dotenv_path=find_dotenv(), override=True)

OPENAI_API_KEY=os.getenv(key="OPENAI_API_KEY")
PINECONE_API_KEY=os.getenv(key="PINECONE_API_KEY")
LANGCHAIN_API_KEY=os.getenv(key="LANGCHAIN_API_KEY")

In [78]:
# colab
from google.colab import userdata

OPENAI_API_KEY=userdata.get('openai_key')
PINECONE_API_KEY=userdata.get('PINECONE_API_KEY')
LANGCHAIN_API_KEY=userdata.get('LANGCHAIN_API_KEY')

# Langsmith Setup

In [79]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langchain-subrata"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY

## Simple LLM → `gpt-3.5-turbo-instruct`

In [80]:
from langchain.llms.openai import OpenAI

llm=OpenAI(
    model_name="gpt-3.5-turbo-instruct",
    openai_api_key=OPENAI_API_KEY,
    temperature=0.9,
    max_tokens=512
  )
print(llm)

OpenAI
Params: {'model_name': 'gpt-3.5-turbo-instruct', 'temperature': 0.9, 'top_p': 1.0, 'frequency_penalty': 0.0, 'presence_penalty': 0.0, 'n': 1, 'logit_bias': {}, 'max_tokens': 512}


In [81]:
prompt="Explain LLM Wrappers and it's use-cases in three sentences."
output=llm(prompt)
print(output)



LLM Wrappers are software components that allow for the execution of code from different programming languages within a single framework. They are commonly used to integrate legacy or third-party code into a new system, and to provide a consistent interface for accessing different programming languages. This can save time and resources by avoiding the need for rewriting or recreating existing code.


In [8]:
prompt_token_length=llm.get_num_tokens(prompt)
print(f"Prmopt token length: {prompt_token_length}")

Prmopt token length: 16


In [9]:
# generate multiple prompts
prompt1="... is the city of joy."
prompt2="Top 10 richest persons are ..."
output=llm.generate([prompt1, prompt2])
print(output)

generations=[[Generation(text="\n\nKnown as the cultural capital of India, Kolkata (formerly known as Calcutta) is a bustling metropolis filled with history, art, and a vibrant mix of cultures. It is located on the eastern bank of the Hooghly River and is the capital of the state of West Bengal. Despite being one of the most populated cities in India, Kolkata manages to maintain its old-world charm, making it a unique and enchanting place to visit.\n\nOne of the main reasons Kolkata is known as the city of joy is because of its warm and welcoming people. The locals, known as Kolkatans, are known for their friendliness and hospitality, making visitors feel at home. The city also has a laid-back and relaxed atmosphere, making it a perfect place to escape the chaos and hustle of other major cities in India.\n\nKolkata is also known for its rich history and cultural heritage. It was the capital of British India until 1911 and still bears the imprint of its colonial past in its architecture

In [10]:
print(len(output.generations), output.generations)

2 [[Generation(text="\n\nKnown as the cultural capital of India, Kolkata (formerly known as Calcutta) is a bustling metropolis filled with history, art, and a vibrant mix of cultures. It is located on the eastern bank of the Hooghly River and is the capital of the state of West Bengal. Despite being one of the most populated cities in India, Kolkata manages to maintain its old-world charm, making it a unique and enchanting place to visit.\n\nOne of the main reasons Kolkata is known as the city of joy is because of its warm and welcoming people. The locals, known as Kolkatans, are known for their friendliness and hospitality, making visitors feel at home. The city also has a laid-back and relaxed atmosphere, making it a perfect place to escape the chaos and hustle of other major cities in India.\n\nKolkata is also known for its rich history and cultural heritage. It was the capital of British India until 1911 and still bears the imprint of its colonial past in its architecture and landm

In [11]:
print(output.generations[0][0])

text="\n\nKnown as the cultural capital of India, Kolkata (formerly known as Calcutta) is a bustling metropolis filled with history, art, and a vibrant mix of cultures. It is located on the eastern bank of the Hooghly River and is the capital of the state of West Bengal. Despite being one of the most populated cities in India, Kolkata manages to maintain its old-world charm, making it a unique and enchanting place to visit.\n\nOne of the main reasons Kolkata is known as the city of joy is because of its warm and welcoming people. The locals, known as Kolkatans, are known for their friendliness and hospitality, making visitors feel at home. The city also has a laid-back and relaxed atmosphere, making it a perfect place to escape the chaos and hustle of other major cities in India.\n\nKolkata is also known for its rich history and cultural heritage. It was the capital of British India until 1911 and still bears the imprint of its colonial past in its architecture and landmarks. The city 

In [12]:
# access only the first prompt result
print(output.generations[0][0].text)



Known as the cultural capital of India, Kolkata (formerly known as Calcutta) is a bustling metropolis filled with history, art, and a vibrant mix of cultures. It is located on the eastern bank of the Hooghly River and is the capital of the state of West Bengal. Despite being one of the most populated cities in India, Kolkata manages to maintain its old-world charm, making it a unique and enchanting place to visit.

One of the main reasons Kolkata is known as the city of joy is because of its warm and welcoming people. The locals, known as Kolkatans, are known for their friendliness and hospitality, making visitors feel at home. The city also has a laid-back and relaxed atmosphere, making it a perfect place to escape the chaos and hustle of other major cities in India.

Kolkata is also known for its rich history and cultural heritage. It was the capital of British India until 1911 and still bears the imprint of its colonial past in its architecture and landmarks. The city is home to m

### Generate Multiple Responses for Single Prompt

In [13]:
# asked llm to generate multiple responses
prompt="Write tagline for a software company that does outsourcing."
outputs=llm.generate([prompt], n=2)
print(outputs)

generations=[[Generation(text='\n\n"Empowering Your Business, Globally. Partner with us for Efficient Outsourcing Solutions."', generation_info={'finish_reason': 'stop', 'logprobs': None}), Generation(text='\n"Outsource with ease, excel with our software."', generation_info={'finish_reason': 'stop', 'logprobs': None})]] llm_output={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 11, 'total_tokens': 42}, 'model_name': 'gpt-3.5-turbo-instruct'} run=[RunInfo(run_id=UUID('f1c8353f-ed77-4970-ad70-3a3d27086b2d'))]


In [14]:
print(len(outputs.generations[0]), outputs.generations)

2 [[Generation(text='\n\n"Empowering Your Business, Globally. Partner with us for Efficient Outsourcing Solutions."', generation_info={'finish_reason': 'stop', 'logprobs': None}), Generation(text='\n"Outsource with ease, excel with our software."', generation_info={'finish_reason': 'stop', 'logprobs': None})]]


In [15]:
for index,output in enumerate(outputs.generations[0]):
  print(output.text)



"Empowering Your Business, Globally. Partner with us for Efficient Outsourcing Solutions."

"Outsource with ease, excel with our software."


### Generate Multiple Responses from Multiple Prompts

In [16]:
# asked llm to generate multiple responses
prompt1="Write tagline for an AI Company."
prompt2="Write a tagline for a Sweet Company."
outputs=llm.generate([prompt1,prompt2] * 4)

print(outputs)
print(len(outputs.generations[0]), outputs.generations)

generations=[[Generation(text='\n\n"Empowering a smarter tomorrow with intelligent solutions: Unleash the power of AI with us."', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\n"Satisfy Your Cravings with Every Bite: Indulge in Our Sweet Treats!"', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\n"Empowering the Future with Intelligent Technology."', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\n"Indulge in a world of sweetness with every bite." ', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n"Empowering the future through intelligent technology"', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\r\n\n"Satisfy your cravings with every bite - Indulge in sweetness with our treats."', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\n"Empowering the Future with Intelligent 

In [17]:
print(outputs.generations[0])

[Generation(text='\n\n"Empowering a smarter tomorrow with intelligent solutions: Unleash the power of AI with us."', generation_info={'finish_reason': 'stop', 'logprobs': None})]


In [18]:
for index,output in enumerate(outputs.generations):
  print(output[0].text)



"Empowering a smarter tomorrow with intelligent solutions: Unleash the power of AI with us."


"Satisfy Your Cravings with Every Bite: Indulge in Our Sweet Treats!"


"Empowering the Future with Intelligent Technology."


"Indulge in a world of sweetness with every bite." 

"Empowering the future through intelligent technology"


"Satisfy your cravings with every bite - Indulge in sweetness with our treats."


"Empowering the Future with Intelligent Innovation" 


"Satisfy Your Sweet Tooth with Our Delectable Treats!"


## Chat Models → `gpt-3.5-turbo, gpt-4o, gpt-4o mini`

* **`AIMessage:`** AIMessage is returned from a chat model as a response to a prompt. This message represents the output of the model and consists of both
the raw output as returned by the model together standardized fields
(e.g., tool calls, usage metadata) added by the LangChain framework.

* **`HumanMessage:`** HumanMessages are messages that are passed in from a human to the model.

* **`SystemMessage:`** Message for priming AI behavior. The system message is usually passed in as the first of a sequence of input messages.

  ```
  messages = [
              SystemMessage(
                  content="You are a helpful assistant! Your name is Bob."
              ),
              HumanMessage(
                  content="What is your name?"
              )
          ]
  ```

In [82]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain.chat_models import ChatOpenAI

In [83]:
%%time
chat=ChatOpenAI(
    model_name="gpt-4o",
    openai_api_key=OPENAI_API_KEY,
    temperature=0,
  )
messages=[
          SystemMessage(
            content="You are an expert AI/LLM Engineer who teaches using the Richard Feynman Technique."
          ),
          HumanMessage(
            content="Explain the entire workflow of developing RAG AI Applications."
          )
        ]
print(messages)
output=chat(messages)
print(output)

[SystemMessage(content='You are an expert AI/LLM Engineer who teaches using the Richard Feynman Technique.'), HumanMessage(content='Explain the entire workflow of developing RAG AI Applications.')]
content="Sure! The Richard Feynman Technique involves breaking down complex concepts into simpler, understandable parts. Let's apply this to explain the workflow of developing Retrieval-Augmented Generation (RAG) AI applications.\n\n### Step 1: Understanding RAG AI Applications\n**RAG AI Applications** combine two main components:\n1. **Retrieval**: Fetching relevant information from a large dataset or knowledge base.\n2. **Generation**: Using a language model to generate coherent and contextually appropriate responses based on the retrieved information.\n\n### Step 2: Breaking Down the Workflow\n\n#### 1. Problem Definition\n- **Objective**: Clearly define what problem the RAG application aims to solve. For example, answering customer queries using a company's internal documents.\n- **Scope

In [84]:
print(output.content)

Sure! The Richard Feynman Technique involves breaking down complex concepts into simpler, understandable parts. Let's apply this to explain the workflow of developing Retrieval-Augmented Generation (RAG) AI applications.

### Step 1: Understanding RAG AI Applications
**RAG AI Applications** combine two main components:
1. **Retrieval**: Fetching relevant information from a large dataset or knowledge base.
2. **Generation**: Using a language model to generate coherent and contextually appropriate responses based on the retrieved information.

### Step 2: Breaking Down the Workflow

#### 1. Problem Definition
- **Objective**: Clearly define what problem the RAG application aims to solve. For example, answering customer queries using a company's internal documents.
- **Scope**: Determine the scope of the application, including the types of questions it should handle and the sources of information it will use.

#### 2. Data Collection and Preparation
- **Data Sources**: Identify and gather

## PromptTemplate

In [22]:
from langchain import PromptTemplate
from langchain_openai import OpenAI

In [23]:
%%time
template="""You are an expert {designation} who teaches using the Richard Feynman Technique. Keep it under {word_count} words."""

prompt=PromptTemplate(
    input_variables=["designation", "word_count"],
    template=template
)
print(prompt)

llm=OpenAI(
    model_name="gpt-3.5-turbo-instruct",
    openai_api_key=OPENAI_API_KEY,
    temperature=0.9
  )
print(llm)

output = llm.invoke(prompt.format(designation="AI Engineer", word_count=100))
print(output)

input_variables=['designation', 'word_count'] template='You are an expert {designation} who teaches using the Richard Feynman Technique. Keep it under {word_count} words.'
OpenAI
Params: {'model_name': 'gpt-3.5-turbo-instruct', 'temperature': 0.9, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'logit_bias': {}, 'seed': None, 'logprobs': None, 'max_tokens': 256}


The Richard Feynman Technique is a powerful method for deepening your understanding of complex topics. As an AI Engineer, I will guide you through breaking down concepts into simple, bite-sized pieces, and then explaining them in a clear and concise manner. By continuously asking "why?" and testing your knowledge through practice problems and simulations, you will master the material and be able to apply it confidently. My goal is to help you not only learn the subject, but also develop critical thinking skills and a strong foundation for future learning. Let's dive in!
CPU times: user 150 ms, sys: 7.05 ms,

## Chains

In [24]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain

In [25]:
template="""You are an expert {designation} who teaches using the Richard Feynman Technique. {question}. Keep it under {word_count} words."""
prompt=PromptTemplate(
    input_variables=["designation", "question", "word_count"],
    template=template
)

# Create a runnable sequence
chain = prompt | llm
print("Chain:\n",chain)

output=chain.invoke(input={
    "designation":"AI Engineer specialized in advanced Retrieval Augmented Generation (RAG) with Large Language Models.",
    "question":"Teach about the advanced methods of RAG",
    "word_count":500
})
print("Output:\n",output)

Chain:
 first=PromptTemplate(input_variables=['designation', 'question', 'word_count'], template='You are an expert {designation} who teaches using the Richard Feynman Technique. {question}. Keep it under {word_count} words.') last=OpenAI(client=<openai.resources.completions.Completions object at 0x780317d6c2e0>, async_client=<openai.resources.completions.AsyncCompletions object at 0x780317d3dae0>, temperature=0.9, openai_api_key=SecretStr('**********'), openai_proxy='')
Output:
 

Retrieval Augmented Generation (RAG) is an advanced method of utilizing Large Language Models (LLMs) for natural language generation tasks. This technique combines the strengths of both retrieval and generation models to create more robust and accurate outputs.

At its core, RAG utilizes a pre-trained LLM, such as GPT-3 or BERT, as a retrieval model to generate a set of relevant documents or passages from a large dataset. These documents act as knowledge sources for the generation model, allowing it to gener

## Sequential Chains
Sequential Chains are Complex Chains.

In [26]:
from langchain_openai import OpenAI,ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain
from langchain.schema.output_parser import StrOutputParser

In [27]:
# output parser
struct_parser=StrOutputParser()
print(struct_parser)

llm1=ChatOpenAI(
    name="gpt-3.5-turbo",
    api_key=OPENAI_API_KEY,
    temperature=0.9,
    max_tokens=1024
)

prompt1=PromptTemplate(
    input_variables=["input"],
    template="""Solve the math expression: {input}"""
)

chain1=prompt1|llm1|struct_parser
print(chain1)

llm2=ChatOpenAI(
    name="gpt-4o",
    api_key=OPENAI_API_KEY,
    temperature=0.2,
    max_tokens=1024
)

prompt2=PromptTemplate(
    input_variables=["input"],
    template="""Solve the math expression: {input}"""
)

chain2=prompt2|llm2|struct_parser
print(chain2)

sequential_chain=chain1|chain2

output=sequential_chain.invoke(input={
    "input": "5.1**7.3"
})

print(output)


first=PromptTemplate(input_variables=['input'], template='Solve the math expression: {input}') middle=[ChatOpenAI(name='gpt-3.5-turbo', client=<openai.resources.chat.completions.Completions object at 0x7803283682b0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x780328368c40>, temperature=0.9, openai_api_key=SecretStr('**********'), openai_proxy='', max_tokens=1024)] last=StrOutputParser()
first=PromptTemplate(input_variables=['input'], template='Solve the math expression: {input}') middle=[ChatOpenAI(name='gpt-4o', client=<openai.resources.chat.completions.Completions object at 0x780328398f40>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x78032839a890>, temperature=0.2, openai_api_key=SecretStr('**********'), openai_proxy='', max_tokens=1024)] last=StrOutputParser()
There is nothing to solve in the given number 3616.00395685. It is already a decimal number.


In [28]:
output

'There is nothing to solve in the given number 3616.00395685. It is already a decimal number.'

## Agents

In [29]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools import PythonREPLTool
from langchain_openai import ChatOpenAI

In [85]:
llm=ChatOpenAI(
    model="gpt-4o",
    api_key=OPENAI_API_KEY,
    temperature=0.9,
    max_tokens=1024
)
agent_executor=create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True
)

agent_executor.invoke(input="Calculate the square root of the factorial of 20 and display it with 4 decimal places.")



> Entering new AgentExecutor chain...


To calculate the square root of the factorial of 20 and display it with 4 decimal places, I need to follow these steps:
1. Calculate the factorial of 20.
2. Compute the square root of the result from step 1.
3. Format the result to 4 decimal places.

Let's perform these steps in the Python REPL.

Action: Python_REPL
Action Input: 
```python
import math

# Calculate factorial of 20
factorial_20 = math.factorial(20)

# Calculate square root of factorial_20
sqrt_factorial_20 = math.sqrt(factorial_20)

# Format the result to 4 decimal places
formatted_result = f"{sqrt_factorial_20:.4f}"
print(formatted_result)
```
Observation: 1559776268.6285

Thought:I now know the final answer.

Final Answer: 1559776268.6285

> Finished chain.


{'input': 'Calculate the square root of the factorial of 20 and display it with 4 decimal places.',
 'output': '1559776268.6285'}

## Pinecone Vector DB
Diving into Vector Databases.

In [86]:
PINECONE_API_KEY=userdata.get(key="PINECONE_API_KEY")

In [87]:
from pinecone import Pinecone, ServerlessSpec

In [88]:
pinecone=Pinecone(
    api_key=PINECONE_API_KEY
  )

print(pinecone.list_indexes())

UnauthorizedException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'X-Cloud-Trace-Context': 'b9b08c656fbc75c9200b2b922ca1bef1', 'Date': 'Thu, 01 Aug 2024 10:46:29 GMT', 'Content-Type': 'text/html', 'Server': 'Google Frontend', 'Content-Length': '15', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: Invalid API Key


### Pinecone Indexes

In [34]:
pinecone.list_indexes()

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 1536,
              'host': 'langchain-pinecone-0390e5b.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'langchain-pinecone',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

### Create an Index

In [35]:
index_name="langchain-pinecone"

def create_index(index_name:str, dimension:int=512, metric:str="cosine"):
  if index_name not in pinecone.list_indexes():
    print(f"Creating index: {index_name}...")
    pinecone.create_index(
        name=index_name,
        dimension=dimension,
        metric=metric,
        spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
      )
    )
    print("Done")
  else:
    print(f"Index: {index_name} already exists")

# create index
create_index(index_name=index_name, dimension=1536, metric="cosine")

Creating index: langchain-pinecone...


PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'x-pinecone-api-version': '2024-07', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'X-Cloud-Trace-Context': '6d2b14593e8dcfbeef4a0b6482213ef9', 'Date': 'Thu, 01 Aug 2024 10:28:19 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [36]:
# List all indexes
print(pinecone.list_indexes())

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 1536,
              'host': 'langchain-pinecone-0390e5b.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'langchain-pinecone',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}


In [37]:
# Describe the newly created index
pinecone.describe_index(index_name)

{'deletion_protection': 'disabled',
 'dimension': 1536,
 'host': 'langchain-pinecone-0390e5b.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'langchain-pinecone',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'}}

### Selecting an Index

In [38]:
index_name="langchain-pinecone"
index=pinecone.Index(index_name)
print(index.describe_index_stats())

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 5}},
 'total_vector_count': 5}


### Deleting an Index

In [154]:
# index_name="langchain-pinecone"

# def delete_index(index_name:str):
#   if index_name in pinecone.list_indexes()[0]["name"]:
#     print(f"Deleting index: {index_name}...")
#     pinecone.delete_index(index_name)
#     print("Done")
#   else:
#     print(f"Index: {index_name} does not exist")

# delete_index(index_name=index_name)

Deleting index: langchain-pinecone...
Done


In [39]:
print(pinecone.list_indexes())

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 1536,
              'host': 'langchain-pinecone-0390e5b.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'langchain-pinecone',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}


### Insert vectors to Pinecone Index with `upsert`

In [40]:
import random

vectors=[[random.random() for _ in range(1536)] for _ in range(5)]
vector_ids=list("abcde")

print(len(vectors), len(vectors[0]))
print(vectors)
print(vector_ids)

5 1536
[[0.8929301500981848, 0.5754327377203153, 0.9202232470744425, 0.7615632434540874, 0.6007481912380226, 0.7912440111876042, 0.546200431190386, 0.4797403759418233, 0.4920965778894095, 0.6225527046558285, 0.1449834537094683, 0.049379253507784604, 0.5685944985032777, 0.6206023134649027, 0.7315847963772467, 0.16499466356148706, 0.4035393560989887, 0.4951402352922658, 0.5954142345024065, 0.12368120768933755, 0.9585031380785025, 0.7842297626422593, 0.6292924950361694, 0.5276200981146523, 0.9989133087163392, 0.16731928333846013, 0.34736794817408745, 0.91718130256841, 0.037404967155481805, 0.6764452832779301, 0.09664289518153513, 0.4638902144451026, 0.7342854187609567, 0.4182494296625915, 0.04428516404458627, 0.15746866748181176, 0.4919217727961164, 0.6131387333951926, 0.6246656483769357, 0.3465421120644734, 0.8758443177639164, 0.025513997306681913, 0.6796983532196246, 0.7391179212438064, 0.4263107878068886, 0.8443144320745011, 0.23888157403387233, 0.8358081352807888, 0.1921954769156119, 

In [41]:
# Prepare the vectors for upsert
vectors_for_upsert = [
    (id, vec, {}) for id, vec in zip(vector_ids, vectors)
]
print(vectors_for_upsert)

[('a', [0.8929301500981848, 0.5754327377203153, 0.9202232470744425, 0.7615632434540874, 0.6007481912380226, 0.7912440111876042, 0.546200431190386, 0.4797403759418233, 0.4920965778894095, 0.6225527046558285, 0.1449834537094683, 0.049379253507784604, 0.5685944985032777, 0.6206023134649027, 0.7315847963772467, 0.16499466356148706, 0.4035393560989887, 0.4951402352922658, 0.5954142345024065, 0.12368120768933755, 0.9585031380785025, 0.7842297626422593, 0.6292924950361694, 0.5276200981146523, 0.9989133087163392, 0.16731928333846013, 0.34736794817408745, 0.91718130256841, 0.037404967155481805, 0.6764452832779301, 0.09664289518153513, 0.4638902144451026, 0.7342854187609567, 0.4182494296625915, 0.04428516404458627, 0.15746866748181176, 0.4919217727961164, 0.6131387333951926, 0.6246656483769357, 0.3465421120644734, 0.8758443177639164, 0.025513997306681913, 0.6796983532196246, 0.7391179212438064, 0.4263107878068886, 0.8443144320745011, 0.23888157403387233, 0.8358081352807888, 0.1921954769156119, 0

In [42]:
vectors_for_upsert[0][2]

{}

In [43]:
index_name="langchain-pinecone"
index=pinecone.Index(index_name)

# upserting vectors with ids to the pinecone indexes
print("No. of vectors inserted...")
index.upsert(
    vectors=vectors_for_upsert,
)

No. of vectors inserted...


{'upserted_count': 5}

### Update Vectors with `upsert`

In [44]:
# updating the vector indexes
index.upsert(
    vectors=[("a",[0.3]*1536, {})]
)

{'upserted_count': 1}

### Fetching a Vector by ID

In [45]:
index_name="langchain-pinecone"
index=pinecone.Index(index_name)
print(index.fetch(ids=["a","b"]))

{'namespace': '',
 'usage': {'read_units': 1},
 'vectors': {'a': {'id': 'a',
                   'values': [0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
             

### Deleting a Vector by ID

In [170]:
# index.delete(ids=["a"])

{}

In [46]:
index.fetch(ids=["a"])

{'namespace': '',
 'usage': {'read_units': 1},
 'vectors': {'a': {'id': 'a',
                   'values': [0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
             

In [47]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 5}},
 'total_vector_count': 5}

### Delete all the Vectors from the Index

In [173]:
# index.delete(delete_all=True)

{}

### Insert the Vectors again

In [48]:
index_name="langchain-pinecone"
index=pinecone.Index(index_name)

# upserting vectors with ids to the pinecone indexes
print("No. of vectors inserted...")
index.upsert(
    vectors=vectors_for_upsert,
)

No. of vectors inserted...


{'upserted_count': 5}

### Querying the Pinecone Index

In [49]:
# Create a single query vector
query_vector = [random.random() for _ in range(1536)]
print("Query vector:", query_vector)

Query vector: [0.48332740471486035, 0.45885024897548476, 0.8989086622710301, 0.28995850094678743, 0.5033281488136737, 0.3153129950785898, 0.10799037800055389, 0.2857537967538396, 0.49574010947856595, 0.1710370951552871, 0.7482141598196451, 0.29793590996458363, 0.3754618767014313, 0.18640078565860974, 0.28747953648635793, 0.25296054582076377, 0.4433239611630717, 0.4050471857620437, 0.06971545036813209, 0.40329624730468816, 0.5239869256870021, 0.4538480132401539, 0.06866173790584595, 0.6517826557951263, 0.8782847854649515, 0.7143002593123632, 0.4827282054797709, 0.555912152608886, 0.18587299486416353, 0.5092397825210806, 0.4373738858821621, 0.6678228902550555, 0.9577635835056316, 0.26458159424152494, 0.5766195241461995, 0.18091122310964214, 0.13602304723239933, 0.3986509275694531, 0.8199660629182831, 0.3792476081266213, 0.9096678240299468, 0.4313296192817848, 0.35162743861967816, 0.44172104943234913, 0.788934166140902, 0.5683677357294183, 0.7827876567504762, 0.6059245900696275, 0.0749252

In [50]:
try:
    query_response = index.query(
        vector=query_vector,  # Changed from 'queries' to 'vector'
        top_k=3,
        include_metadata=True
    )
    print("Query response:", query_response)
except pinecone.PineconeException as e:
    print("Pinecone error:", str(e))

Query response: {'matches': [{'id': 'a', 'score': 0.768186, 'values': []},
             {'id': 'b', 'score': 0.761339486, 'values': []},
             {'id': 'c', 'score': 0.758001328, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 6}}


## Splitting & Embedding

In [51]:
!pip install python-docx -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 5.6 MB/s eta 0:00:00


### Create Chunks

In [55]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter()

# Create a text splitter
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
)

print(text_splitter)

In [56]:
from docx import Document

# Load the document
doc = Document("/content/accounting.docx")

# Extract text from the document
full_text = []
for para in doc.paragraphs:
    full_text.append(para.text)

# Join all paragraphs into a single string
text = '\n'.join(full_text)

# Print the first 500 characters of the text
print(text[:500])

CHAPTER 1
UNIT - 1 
MEANING AND SCOPE OF ACCOUNTING
INTRODUCTION

Every individual performs some kind of economic activity. A salaried person gets salary and spends to buy provisions and clothing, for children's education, construction of house, etc. A sports club formed by a group of individuals, a business run by an individual or a group of individuals, a company running a business in telecom sector, a local authority like Calcutta Municipal Corporation, Delhi Development Authority, Government


In [57]:
type(text)

str

In [58]:
# chunks
chunks=text_splitter.create_documents([text])
print(len(chunks))

10321


In [59]:
type(chunks)

list

In [60]:
chunks[:5]

[Document(page_content='CHAPTER 1\nUNIT - 1 \nMEANING AND SCOPE OF ACCOUNTING\nINTRODUCTION'),
 Document(page_content='Every individual performs some kind of economic activity. A salaried person gets salary and spends'),
 Document(page_content="salary and spends to buy provisions and clothing, for children's education, construction of house,"),
 Document(page_content='of house, etc. A sports club formed by a group of individuals, a business run by an individual or a'),
 Document(page_content='an individual or a group of individuals, a company running a business in telecom sector, a local')]

=

In [61]:
chunks[0]

Document(page_content='CHAPTER 1\nUNIT - 1 \nMEANING AND SCOPE OF ACCOUNTING\nINTRODUCTION')

In [62]:
chunks[0].page_content

'CHAPTER 1\nUNIT - 1 \nMEANING AND SCOPE OF ACCOUNTING\nINTRODUCTION'

In [63]:
len(chunks[0].page_content)

64

### Embedding Cost

In [64]:
def print_embedding_cost(texts:str                    ):
    import tiktoken
    enc = tiktoken.encoding_for_model('text-embedding-ada-002')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    print(f'Total Tokens: {total_tokens}')
    print(f'Embedding Cost in USD: {total_tokens / 1000 * 0.0001:.6f}')
    return total_tokens

print_embedding_cost(texts=chunks)

Total Tokens: 173218
Embedding Cost in USD: 0.017322


173218

### OpenAI Embeddings

In [89]:
from langchain.embeddings import OpenAIEmbeddings

In [90]:
embeddings=OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY
)

In [91]:
query_vector=embeddings.embed_query(text="abc")
print(len(query_vector))

1536


In [92]:
# embeddings for the first chunk
query_vector=embeddings.embed_query(text=chunks[0].page_content)
print(len(query_vector))
print(query_vector)

1536
[0.018641443097154948, -0.0336356446144922, 0.01747635301999808, -0.02014846079185124, -0.005999581156828709, 0.007573085967180905, 0.009782958342297669, 0.0158553563483802, 0.0006905850299125014, -0.036320420235444, 0.004445072066528, 0.01764098476686354, -0.008655859705243775, 0.020199117287085247, 0.0037802109216862017, -0.008301267032573315, 0.013525177125388997, -0.008174626725810825, -0.019629235440992782, 0.0022874392021213067, -0.03310375513982524, 0.019160666585368337, -0.036396401253004876, 0.01800824063201997, 0.0051827516904380515, 0.013385872694818007, 0.022478640946164974, -0.022465976822356473, 0.017324383534296054, -0.014601619267208883, -0.00643015764102765, -0.024099635755137787, -0.00023171205676745087, -0.005382209777776895, 0.0068322407779799945, 0.008168294663906575, 0.0007903141899972394, -9.354557864094073e-05, 0.012391746426430853, -0.015792037591982756, 0.013702472996062962, 0.015209491622081787, -0.016045318205507732, 0.009523345666868441, -0.02565731180

### Inserting all the Embeddings into Pinecone Index

In [93]:
index=pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 5}},
 'total_vector_count': 5}

In [108]:
pinecone.list_indexes()[0]["name"]

'langchain-pinecone-rag'

In [132]:
from pinecone import Pinecone
pinecone=Pinecone(
    api_key=PINECONE_API_KEY
)
# deleting all the indexes
indexes=pinecone.list_indexes()
for i in range(len(indexes)):
  print("Deleting all indexes")
  pinecone.delete_index(indexes[i]["name"])
  print("Done")

In [133]:
pinecone.list_indexes()

{'indexes': []}

In [134]:
# create a new index
index_name="langchain-subrata"
create_index(
    index_name=index_name,
    dimension=1536,
    metric="cosine"
  )
print("Done")
pinecone.list_indexes()

Creating index: langchain-subrata...
Done
Done


{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 1536,
              'host': 'langchain-subrata-0390e5b.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'langchain-subrata',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [137]:
!pip install langchain_pinecone -q

In [148]:
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
import os

# Set the Pinecone API key as an environment variable
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

# Initialize Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

# Create OpenAI embeddings
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# Create Pinecone vector store
docsearch = PineconeVectorStore.from_documents(
    documents=chunks,
    embedding=embeddings,
    index_name=index_name,
)

print(type(docsearch))

<class 'langchain_pinecone.vectorstores.PineconeVectorStore'>


## Asking Questions(Similarity Search)

In [149]:
query = "Explain the Procedural aspects of Accounting"
docs = docsearch.similarity_search(query)
print(docs)
print(docs[0].page_content)

[Document(page_content='Procedural aspects of Accounting'), Document(page_content='SEQUENCE OF ACCOUNTING PROCEDURE OR THE ACCOUNTING CYCLE'), Document(page_content='and procedures of accounting serving as an explanation of current practices and as a guide for'), Document(page_content='What has been done so far shows that the accounting process in the following order:')]
Procedural aspects of Accounting


In [152]:
docs

[Document(page_content='Procedural aspects of Accounting'),
 Document(page_content='SEQUENCE OF ACCOUNTING PROCEDURE OR THE ACCOUNTING CYCLE'),
 Document(page_content='and procedures of accounting serving as an explanation of current practices and as a guide for'),
 Document(page_content='What has been done so far shows that the accounting process in the following order:')]

In [155]:
print("Retrieved documents:")
for i, doc in enumerate(docs, 1):
    print(f"\nDocument {i}:")
    print(doc.page_content)
    print("-" * 50)

Retrieved documents:

Document 1:
Procedural aspects of Accounting
--------------------------------------------------

Document 2:
SEQUENCE OF ACCOUNTING PROCEDURE OR THE ACCOUNTING CYCLE
--------------------------------------------------

Document 3:
and procedures of accounting serving as an explanation of current practices and as a guide for
--------------------------------------------------

Document 4:
What has been done so far shows that the accounting process in the following order:
--------------------------------------------------


## RetrievalQA

In [180]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Set up the language model
gpt4o = ChatOpenAI(
    model="gpt-4",
    api_key=OPENAI_API_KEY,
    temperature=0.9
)

# Set up the retriever
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 5})

# Create the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer concise.\n\nContext: {context}"),
    ("human", "{input}")  # Changed from {question} to {input}
])

# Create the document chain
document_chain = create_stuff_documents_chain(gpt4o, prompt)

# Create the retrieval chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)

print("Retrieval Chain:", retrieval_chain)
print("Retriever:", retriever)

# question1
query = "What are the main procedural aspects of accounting?"
response = retrieval_chain.invoke({"input": query})
print("\nQuery:", query)
print("Response:", response['answer'])

Retrieval Chain: bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['PineconeVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x780312779cc0>, search_kwargs={'k': 5}), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['context', 'input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer concise.\n\nContext: {context}")), HumanMessagePromptTemplate(prompt=PromptT

In [175]:
# question2
query = "Hey! My name is Subrata Mondal. I have completed my UG in Computer Science Engineering. Can you tell me about myself?"
response = retrieval_chain.invoke({"input": query})
print("\nQuery:", query)
print("Response:", response['answer'])


Query: Hey! My name is Subrata Mondal. I have completed my UG in Computer Science Engineering. Can you tell me about myself?
Response: As an AI, I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. Thus, I don't know detailed personal information about you, Subrata Mondal.


In [176]:
# question3
query = "Tell me the distance between the sun and the earth?"
response = retrieval_chain.invoke({"input": query})
print("\nQuery:", query)
print("Response:", response['answer'])


Query: Tell me the distance between the sun and the earth?
Response: The text doesn't provide information on the distance between the sun and the earth.


In [177]:
# question4
query = "Can you tell me about the meaning of Accounting?"
response = retrieval_chain.invoke({"input": query})
print("\nQuery:", query)
print("Response:", response['answer'])


Query: Can you tell me about the meaning of Accounting?
Response: Accounting is the process of identifying, measuring, and communicating. It is a measurement discipline.


In [178]:
# question5
query = "What are the procedural aspects of accounting"
response = retrieval_chain.invoke({"input": query})
print("\nQuery:", query)
print("Response:", response['answer'])


Query: What are the procedural aspects of accounting
Response: The context does not provide specific details on the procedural aspects of accounting.


In [181]:
# question5
query = "Tell me about the basic function of accounting?"
response = retrieval_chain.invoke({"input": query})
print("\nQuery:", query)
print("Response:", response['answer'])


Query: Tell me about the basic function of accounting?
Response: The basic function of accounting is to provide quantitative information, primarily of a financial nature. It also involves book-keeping, which lays the foundation for the accounting process.
